In [1]:
import pandas as pd
import tqdm
import requests
from sqlalchemy.orm import Session

from ragas.metrics import (
    Faithfulness,
    AnswerRelevancy,
    ContextPrecision,
    ContextRecall,
    SummarizationScore,
)
from ragas import evaluate

In [2]:
test_data_path = "/Users/ekaterina.baru/projects/SpicyRAG/data/v2_ragas_npa_dataset_firstPart.xlsx"

In [3]:
# project:
#   name: "SpicyRag"
# 
# database:
#   user: "user"
#   password: "password"
#   name: "spicyragdb"
#   host: "localhost"
#   port: 5432
# 
# logging:
#   level: "INFO"
# 
# server:
#   host: "0.0.0.0"
#   port: 8000
# 
# llm:
#   base_url: "https://integrate.api.nvidia.com/v1"
#   api_key: "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"
#   model: "meta/llama-3.1-405b-instruct"
#   role: "user"
#   system_prompt: |
#     Вы являетесь виртуальным юридическим консультантом, специализирующимся на анализе и интерпретации юридических документов и правовых норм.
#     Ваша задача — помочь пользователю, используя предоставленный контекст, чтобы дать точный и обоснованный ответ на его юридический вопрос.
#     Контекст может содержать выдержки из законов, нормативных актов, судебных решений, договоров или других юридических текстов.
# 
#     Инструкции:
#       1. Внимательно проанализируй контекст и определите, какие части контекста наиболее релевантны, приводи всю полезную информацию из контекста
#       2. Используй релевантные фрагменты контекста (не пиши к чему обращаешься, не упоминай пунктов) для создания полного и юридически обоснованного ответа. Приводи цитаты из контекста в соответствии
#       3. После ответа напиши вывод о написанной тобой информации, начиная со слова Таким образом
# 
#     ВАЖНО: не упоминай, что ты работаешь с контекстом! Отвечай структурно и по пунктам.
#   temperature: 0.15
#   top_p: 0.7
#   max_tokens: 10000
# 
# llm_rewriter:
#   base_url: "https://integrate.api.nvidia.com/v1"
#   api_key: "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"
#   model: "meta/llama-3.1-8b-instruct"
#   role: "user"
#   system_prompt: |
#     Вы - полезный помощник, который генерирует несколько поисковых запросов на основе одного входного запроса.
#     Выполните расширение запроса. Если существует несколько распространенных способов формулировки пользовательского вопроса или общих синонимов для ключевых слов в вопросе, обязательно верните несколько версий запроса с разными формулировками.
#     Если в тексте есть сокращения или слова, которые вам незнакомы, не пытайтесь их перефразировать.
#     Верните только 3 разных варианта вопроса.
#   temperature: 0.2
#   top_p: 0.7
#   max_tokens: 1024
# 
# embedding_model:
#   name: "paraphrase-MiniLM-L6-v2"
#   dimension: 384
# 
# data_sources:
#   excel_file: "./data/v2_ragas_npa_dataset_firstPart.xlsx"
#   text_file: "./data/hmao_npa.txt"
#   text_separator: "\n\n"
# 
# data_processing:
#   chunker:
#     py_class: interface.chunker.RecursiveCharacterTextSplitterChunker
#     kwargs:
#       chunk_size: 2048
#       chunk_overlap: 128
#       separators:
#         - '\n#{1,6} '
#         - '```\n'
#         - '\n\\*\\*\\*+\n'
#         - '\n---+\n'
#         - '\n___+\n'
#         - '\n\n'
#         - '\n'
#         - ' '
#         - ''
#         - ','
#         - '|'
#   top_k: 3
#   similarity_threshold: 0.7

In [4]:
def get_system_responses(questions):
    """
    Send requests to the /ask endpoint and collect responses.
    """
    responses = []
    for question in tqdm.tqdm(questions.itertuples()):
        response = requests.post("http://localhost:8000/ask/", json={"text": question[1]})
        response.raise_for_status()
        response_data = response.json()
        responses.append(
            {
                "question": question[1],
                "response": response_data["llm_response"],
                "ground_truth": question[3],
                "context": response_data["contexts"], 
            }
        )
    return responses

In [5]:
test_data = pd.read_excel(test_data_path)

In [6]:
system_responses = get_system_responses(test_data)

TypeError: string indices must be integers

In [7]:
system_responses

NameError: name 'system_responses' is not defined